In [ ]:
import sys, os
sys.path.extend([os.path.abspath('../')])

In [ ]:
import numpy as np
import sympy as sy
import sparse as sp
import math

In [ ]:
from qgs.params.params import QgParams
from qgs.functions.tendencies import create_tendencies

In [ ]:
from qgs.inner_products.symbolic import AtmosphericSymbolicInnerProducts, OceanicSymbolicInnerProducts
from qgs.tensors.qgtensor import QgsTensor, QgsTensorDynamicT, QgsTensorT4
from qgs.tensors.symbolic_qgtensor import SymbolicTensorLinear, SymbolicTensorDynamicT
from qgs.tensors.symbolic_qgtensor import _shift_dict_keys, _add_to_dict

In [ ]:
model_parameters = QgParams({'n': 1.5}, dynamic_T=True)

In [ ]:
model_parameters.set_atmospheric_channel_fourier_modes(2, 2, mode="symbolic")
# Mode truncation at the wavenumber 2 in the x and at the 
# wavenumber 4 in the y spatial coordinates for the ocean
model_parameters.set_oceanic_basin_fourier_modes(2, 4, mode="symbolic")

In [ ]:
model_parameters.symbolic_params

In [ ]:
# Setting MAOOAM parameters according to the publication linked above
model_parameters.set_params({'kd': 0.0290, 'kdp': 0.0290, 'r': 1.e-7,
                             'h': 136.5, 'd': 1.1e-7})
model_parameters.atemperature_params.set_params({'eps': 0.7, 'hlambda': 15.06})
model_parameters.gotemperature_params.set_params({'gamma': 5.6e8})

In [ ]:
model_parameters.atemperature_params.set_insolation(103., 0)
model_parameters.atemperature_params.set_insolation(103., 1)
model_parameters.gotemperature_params.set_insolation(310., 0)
model_parameters.gotemperature_params.set_insolation(310., 1)

In [ ]:
model_parameters.print_params()

In [ ]:
%%time
# Takes ~5 mins
# atm_ip = AtmosphericSymbolicInnerProducts(model_parameters)
# ocn_ip_stored = OceanicSymbolicInnerProducts(model_parameters, stored=True, return_symbolic=True)
ocn_ip = OceanicSymbolicInnerProducts(model_parameters, stored=False, return_symbolic=True)

In [ ]:
%%time
# Takes ~5 mins
# atm_ip_stored = AtmosphericSymbolicInnerProducts(model_parameters, return_symbolic=True)
atm_ip = AtmosphericSymbolicInnerProducts(model_parameters, stored=False, return_symbolic=True)

In [ ]:
# atm_ip_stored.save_to_file('temp_atm_sym_de.ip')
# ocn_ip_stored.save_to_file('temp_ocn_sym_de.ip')

In [ ]:
atm_loaded = AtmosphericSymbolicInnerProducts(model_parameters, stored=False, return_symbolic=True)
ocn_loaded = OceanicSymbolicInnerProducts(model_parameters, stored=False, return_symbolic=True)

In [ ]:
atm_loaded.load_from_file('temp_atm_sym_de.ip')
ocn_loaded.load_from_file('temp_ocn_sym_de.ip')

In [ ]:
ocn_num = OceanicSymbolicInnerProducts(model_parameters, stored=True)

## Testing the symbolic_qgtensor

In [ ]:
from qgs.functions.symbolic_output import create_symbolic_equations, translate_equations, print_equations, equation_as_function

In [ ]:
f_num, Df, ten_num = create_tendencies(model_parameters, return_qgtensor=True)

In [ ]:
%%time
funcs = create_symbolic_equations(model_parameters, atm_loaded, ocn_loaded)

In [ ]:
ten = SymbolicTensorDynamicT(model_parameters, atm_loaded, ocn_loaded, numerically_test_tensor=False)

In [ ]:
ten.print_tensor(ten.tensor_dic)

In [ ]:
ten.print_tensor(ten_num.tensor.todense())

I the error is comming from the _O and _C terms having some non zeros that should be zero

In [ ]:
ten.print_tensor(ocn_loaded._C)

In [ ]:
ten.print_tensor(ocn_num._O.todense())

In [ ]:
%%time
fx = equation_as_function(funcs, model_parameters, subs=True, language='julia', string_output=True)

In [ ]:
for i in fx:
    print(i)

In [ ]:
from numba import njit

In [ ]:
@njit
def f(t, U):
    #Tendency function of the qgs model
    U_out = np.empty_like(U)
    U_out[0] = -0.0145*U[0] + 0.0145*U[11] + 0.0505574149661188*U[22] + 0.0734121368001177*U[24]
    U_out[1] = -1.24659182237138*U[0]*U[2] - 1.24659182237138*U[11]*U[13] + 0.0145*U[12] - 1.9945469157942*U[14]*U[16] - 2.59615384615385*U[15]*U[18] + 2.59615384615385*U[16]*U[17] - 0.0145*U[1] - 0.00295864958311857*U[21] + 0.115315741885204*U[2] - 1.9945469157942*U[3]*U[5] - 2.59615384615385*U[4]*U[7] + 2.59615384615385*U[5]*U[6]
    U_out[2] = 1.24659182237138*U[0]*U[1] + 1.24659182237138*U[11]*U[12] + 0.0145*U[13] + 1.9945469157942*U[14]*U[15] + 2.59615384615385*U[15]*U[17] + 2.59615384615385*U[16]*U[18] - 0.115315741885204*U[1] + 0.0145*U[25] - 0.0145*U[2] + 1.9945469157942*U[3]*U[4] + 2.59615384615385*U[4]*U[6] + 2.59615384615385*U[5]*U[7]
    U_out[3] = 2.16075915877706*U[12]*U[16] - 2.16075915877706*U[13]*U[15] + 0.0145*U[14] + 4.32151831755411*U[17]*U[20] - 4.32151831755411*U[18]*U[19] + 2.16075915877706*U[1]*U[5] - 0.00216427290094687*U[21] + 0.0238416302768307*U[23] - 2.16075915877706*U[2]*U[4] - 0.0145*U[3] + 4.32151831755411*U[6]*U[9] - 4.32151831755411*U[7]*U[8]
    U_out[4] = -1.21002512891515*U[0]*U[5] - 1.21002512891515*U[11]*U[16] + 2.43*U[12]*U[18] + 0.345721465404329*U[13]*U[14] - 2.43*U[13]*U[17] + 0.0145*U[15] + 2.43*U[1]*U[7] - 0.00449241352700723*U[22] + 0.345721465404329*U[2]*U[3] - 2.43*U[2]*U[6] - 0.0145*U[4] + 0.0599641857803059*U[5]
    U_out[5] = 1.21002512891515*U[0]*U[4] + 1.21002512891515*U[11]*U[15] - 0.345721465404329*U[12]*U[14] - 2.43*U[12]*U[17] - 2.43*U[13]*U[18] + 0.0145*U[16] - 0.345721465404329*U[1]*U[3] - 2.43*U[1]*U[6] + 0.0145*U[26] - 2.43*U[2]*U[7] - 0.0599641857803059*U[4] - 0.0145*U[5]
    U_out[6] = -3.24113873816558*U[0]*U[7] - 3.24113873816558*U[11]*U[18] + 0.675*U[12]*U[16] + 0.675*U[13]*U[15] - 5.18582198106493*U[14]*U[20] + 0.0145*U[17] + 0.675*U[1]*U[5] - 0.000192312222902707*U[21] + 0.675*U[2]*U[4] - 5.18582198106493*U[3]*U[9] - 0.0145*U[6] + 0.0749552322253824*U[7]
    U_out[7] = 3.24113873816558*U[0]*U[6] + 3.24113873816558*U[11]*U[17] - 0.675*U[12]*U[15] + 0.675*U[13]*U[16] + 5.18582198106493*U[14]*U[19] + 0.0145*U[18] - 0.675*U[1]*U[4] + 0.675*U[2]*U[5] + 5.18582198106493*U[3]*U[8] - 0.0749552322253824*U[6] - 0.0145*U[7]
    U_out[8] = -2.65939588772561*U[0]*U[9] - 2.65939588772561*U[11]*U[20] - 2.65939588772561*U[14]*U[18] + 0.0145*U[19] - 0.000431962839135311*U[22] - 2.65939588772561*U[3]*U[7] - 0.0145*U[8] + 0.0576578709426019*U[9]
    U_out[9] = 2.65939588772561*U[0]*U[8] + 2.65939588772561*U[11]*U[19] + 2.65939588772561*U[14]*U[17] + 0.0145*U[20] + 2.65939588772561*U[3]*U[6] - 0.0576578709426019*U[8] - 0.0145*U[9]
    U_out[10] = -0.000510753549879668*U[10]**4 - 0.014593023255814*U[10] - 0.00183644645351737*U[21] - 0.00374635076517544*U[23] + 1.59610484337396e-5*U[29]**4 + 5.17501542233338e-5*U[29]**3*U[30] + 1.72500514077779e-5*U[29]**3*U[32] + 0.00729651162790698*U[29] + 0.00591432955679692*U[30] + 0.00197144318559898*U[32] + 0.000531003677198668
    U_out[11] = 0.00131818181818182*U[0] - 0.00185728563592607*U[10]**3*U[11] - 0.0198572938689218*U[11] - 1.63693875664928*U[12]*U[2] + 1.63693875664928*U[13]*U[1] - 1.30955100531943*U[15]*U[5] + 1.30955100531943*U[16]*U[4] - 3.27387751329857*U[17]*U[7] + 3.27387751329857*U[18]*U[6] - 2.61910201063885*U[19]*U[9] + 2.61910201063885*U[20]*U[8] - 0.00459612863328353*U[22] - 0.00667383061819252*U[24] + 4.43550120954776e-5*U[29]**3*U[31] + 1.7742004838191e-5*U[29]**3*U[33] + 0.00506916671004016*U[31] + 0.00202766668401607*U[33] + 0.000482730615635153
    U_out[12] = -1.6647358298754*U[0]*U[13] - 0.0015418975090707*U[10]**3*U[12] + 1.05320021890077*U[11]*U[2] - 0.0287966213251426*U[12] + 0.0282849932925971*U[13] + 1.68512035024123*U[14]*U[5] + 1.06132075471698*U[15]*U[7] - 2.66357732780065*U[16]*U[3] - 1.06132075471698*U[16]*U[6] + 2.33490566037736*U[17]*U[5] - 2.33490566037736*U[18]*U[4] + 0.00355660377358491*U[1] + 0.000725706501519649*U[21] - 2.04500508591402e-5*U[29]**3*U[30] - 0.00233715902975374*U[30]
    U_out[13] = 1.6647358298754*U[0]*U[12] - 0.0015418975090707*U[10]**3*U[13] - 1.05320021890077*U[11]*U[1] - 0.0282849932925971*U[12] - 0.0287966213251426*U[13] - 1.68512035024123*U[14]*U[4] + 2.66357732780065*U[15]*U[3] - 1.06132075471698*U[15]*U[6] - 1.06132075471698*U[16]*U[7] + 2.33490566037736*U[17]*U[4] + 2.33490566037736*U[18]*U[5] - 0.00355660377358491*U[25] + 4.81842971584593e-5*U[29]**3*U[34] + 0.00355660377358491*U[2] + 0.00550680122860904*U[34]
    U_out[14] = -0.00145929585679905*U[10]**3*U[14] - 1.44050610585137*U[12]*U[5] + 1.44050610585137*U[13]*U[4] - 0.0311378737541528*U[14] - 2.67522562515254*U[15]*U[2] + 2.67522562515254*U[16]*U[1] - 2.88101221170274*U[17]*U[9] + 2.88101221170274*U[18]*U[8] - 5.35045125030509*U[19]*U[7] + 5.35045125030509*U[20]*U[6] + 0.000618363685984819*U[21] - 0.00681189436480877*U[23] - 1.74251833232233e-5*U[29]**3*U[30] + 3.1365329981802e-5*U[29]**3*U[32] - 0.00199145835037292*U[30] + 0.00358462503067126*U[32] + 0.00414285714285714*U[3]
    U_out[15] = -1.35185957626052*U[0]*U[16] - 0.00125723950739611*U[10]**3*U[15] + 0.421071015556554*U[11]*U[5] - 0.45*U[12]*U[7] - 1.63996079743079*U[13]*U[3] + 0.45*U[13]*U[6] + 1.90590038620335*U[14]*U[2] - 0.0368649373881932*U[15] + 0.0230631483770407*U[16] - 2.31923076923077*U[17]*U[2] + 2.31923076923077*U[18]*U[1] + 0.00172785135654124*U[22] - 1.66746568543758e-5*U[29]**3*U[31] - 0.00190568351656843*U[31] + 0.00557692307692308*U[4]
    U_out[16] = 1.35185957626052*U[0]*U[15] - 0.00125723950739611*U[10]**3*U[16] - 0.421071015556554*U[11]*U[4] + 1.63996079743079*U[12]*U[3] + 0.45*U[12]*U[6] + 0.45*U[13]*U[7] - 1.90590038620335*U[14]*U[1] - 0.0230631483770407*U[15] - 0.0368649373881932*U[16] - 2.31923076923077*U[17]*U[1] - 2.31923076923077*U[18]*U[2] - 0.00557692307692308*U[26] + 3.92887346061283e-5*U[29]**3*U[35] + 0.00449016100178891*U[35] + 0.00557692307692308*U[5]
    U_out[17] = -3.421202001397*U[0]*U[18] - 0.00102150709975934*U[10]**3*U[17] + 0.180063263231421*U[11]*U[7] - 0.7875*U[12]*U[5] - 0.7875*U[13]*U[4] + 0.288101221170275*U[14]*U[9] + 1.4625*U[15]*U[2] + 1.4625*U[16]*U[1] - 0.043546511627907*U[17] + 0.0374776161126912*U[18] - 5.47392320223521*U[20]*U[3] + 9.61561114513534e-5*U[21] - 2.70963173883607e-6*U[29]**3*U[30] - 0.00030967357144237*U[30] + 0.00725*U[6]
    U_out[18] = 3.421202001397*U[0]*U[17] - 0.00102150709975934*U[10]**3*U[18] - 0.180063263231421*U[11]*U[6] + 0.7875*U[12]*U[4] - 0.7875*U[13]*U[5] - 0.288101221170275*U[14]*U[8] - 1.4625*U[15]*U[1] + 1.4625*U[16]*U[2] - 0.0374776161126912*U[17] - 0.043546511627907*U[18] + 5.47392320223521*U[19]*U[3] + 0.00725*U[7]
    U_out[19] = -2.75575081119393*U[0]*U[20] - 0.000888267043268988*U[10]**3*U[19] - 0.250522801017629*U[11]*U[9] + 1.00209120407052*U[14]*U[7] - 4.00836481628207*U[18]*U[3] - 0.0473230535894843*U[19] + 0.0325892314023402*U[20] + 0.00024415290907648*U[22] - 2.35620151203137e-6*U[29]**3*U[31] - 0.000269281366471626*U[31] + 0.00819565217391304*U[8]
    U_out[20] = 2.75575081119393*U[0]*U[19] - 0.000888267043268988*U[10]**3*U[20] + 0.250522801017629*U[11]*U[8] - 1.00209120407052*U[14]*U[6] + 4.00836481628207*U[17]*U[3] - 0.0325892314023402*U[19] - 0.0473230535894843*U[20] + 0.00819565217391304*U[9]
    U_out[21] = 2.30554431681946e-7*U[12] + 2.5547312812995e-7*U[14] + 1.41879650265813e-7*U[17] - 2.30554431681946e-7*U[1] - 4.98594961275655e-7*U[21] - 0.000231547500016414*U[22]*U[25] - 0.000393263214313592*U[22]*U[27] - 0.00077917571434095*U[23]*U[26] - 0.00102174928578672*U[23]*U[28] - 0.00156202678582502*U[24]*U[27] - 2.49430228227486e-5*U[25] - 2.5547312812995e-7*U[3] - 1.41879650265813e-7*U[6]
    U_out[22] = 1.27676499203106e-7*U[0] - 1.27676499203106e-7*U[11] + 4.43165421807241e-7*U[15] + 1.84356815471812e-7*U[19] - 0.000297563940911795*U[21]*U[25] + 0.00056941247952257*U[21]*U[27] - 1.45521268835226e-6*U[22] - 0.00185518259457354*U[23]*U[25] - 0.00363689261114416*U[24]*U[26] - 2.49312940023328e-5*U[26] - 4.43165421807241e-7*U[4] - 1.84356815471812e-7*U[8]
    U_out[23] = -4.59275459509595e-7*U[14] - 0.00110122563125871*U[21]*U[26] + 0.0019601816236405*U[21]*U[28] + 0.000385428970940548*U[22]*U[25] - 3.04757790976247e-6*U[23] - 0.00547309138735578*U[24]*U[25] - 2.49117704610167e-5*U[27] + 4.59275459509595e-7*U[3]
    U_out[24] = 5.09747214298137e-8*U[0] - 5.09747214298137e-8*U[11] - 0.0028417174170155*U[21]*U[27] - 0.000594010608460015*U[22]*U[26] + 0.00259237963074834*U[23]*U[25] - 5.2727039845599e-6*U[24] - 2.48844888171634e-5*U[28]
    U_out[25] = -5.43087366736076e-7*U[13] + 0.000529111412123567*U[21]*U[22] + 2.49364240037345e-5*U[21] + 0.00146975392256546*U[22]*U[23] + 0.00288071768822831*U[23]*U[24] - 1.03680315467796e-6*U[25] + 5.43087366736076e-7*U[2]
    U_out[26] = -1.04390781015316e-6*U[16] + 0.00188040062849988*U[21]*U[23] + 0.00423090141412473*U[22]*U[24] + 2.49247013868922e-5*U[22] - 1.99291491029239e-6*U[26] + 1.04390781015316e-6*U[5]
    U_out[27] = -0.000176149545706673*U[21]*U[22] + 0.00440373864266683*U[21]*U[24] + 2.49051881654425e-5*U[23] - 3.58443843007766e-6*U[27]
    U_out[28] = -0.000938435679208949*U[21]*U[23] + 2.4877920928742e-5*U[24] - 5.80838943890229e-6*U[28]
    U_out[29] = 9.12059910499408e-6*U[10]**4 + 4.64878611115251e-5*U[10]**3*U[12] + 1.67415119447806e-5*U[10]**3*U[14] + 9.29757222230502e-6*U[10]**3*U[17] + 0.000521179402000664*U[10] + 0.000664115245473612*U[12] + 0.000239165516521407*U[14] + 0.000132823049094722*U[17] + 1.50119582264922*U[21]*U[35] - 0.750597911324609*U[21]*U[37] - 1.50119582263017*U[22]*U[34] + 1.12589686698056*U[22]*U[36] - 4.50358746792226*U[23]*U[35] + 2.25179373396113*U[23]*U[37] - 2.62709268963613*U[24]*U[34] - 5.62948433490282*U[24]*U[36] + 1.50119582263017*U[25]*U[31] + 2.62709268963613*U[25]*U[33] - 1.50119582264922*U[26]*U[30] + 4.50358746792226*U[26]*U[32] - 1.12589686698056*U[27]*U[31] + 5.62948433490282*U[27]*U[33] + 0.750597911324609*U[28]*U[30] - 2.25179373396113*U[28]*U[32] - 8.14339205803043e-7*U[29]**4 - 1.05879118406788e-22*U[29]**3*U[30] + 4.2351647362715e-22*U[29]**3*U[32] - 0.000260589700996678*U[29] + 5.70773716830428e-5
    U_out[30] = 3.3881317890172e-21*U[10]**4 - 5.31652508387687e-5*U[10]**3*U[12] - 2.75103051082387e-5*U[10]**3*U[14] - 1.06330501677537e-5*U[10]**3*U[17] - 1.06685493772574e-16*U[10] - 0.000759506950146691*U[12] - 0.000393006100803128*U[14] - 0.000151901390029338*U[17] - 1.21682350103801*U[21]*U[35] + 0.608411750519007*U[21]*U[37] + 2.34182350103837*U[22]*U[34] - 1.28761762577863*U[22]*U[36] + 5.15047050311451*U[23]*U[35] - 2.57523525155726*U[23]*U[37] + 2.12944112681652*U[24]*U[34] + 6.43808812889314*U[24]*U[36] - 2.34182350103837*U[25]*U[31] - 2.12944112681652*U[25]*U[33] + 1.21682350103801*U[26]*U[30] - 5.15047050311451*U[26]*U[32] + 1.28761762577863*U[27]*U[31] - 6.43808812889314*U[27]*U[33] - 0.608411750519007*U[28]*U[30] + 2.57523525155726*U[28]*U[32] + 1.32348898008484e-22*U[29]**4 - 3.25735682321217e-6*U[29]**3*U[30] - 8.470329472543e-22*U[29]**3*U[32] - 0.000260589700996678*U[30] - 1.16551733542192e-17
    U_out[31] = 2.78802933171573e-5*U[10]**3*U[11] - 1.54836099362061e-5*U[10]**3*U[15] - 3.09672198724122e-6*U[10]**3*U[19] + 0.000398291670074584*U[11] - 0.000221195408173121*U[15] - 4.42390816346243e-5*U[19] - 1.125*U[21]*U[34] + 0.375*U[21]*U[36] + 1.875*U[23]*U[34] + 2.25*U[24]*U[35] + 1.125*U[25]*U[30] - 1.875*U[25]*U[32] - 2.25*U[26]*U[33] - 0.375*U[27]*U[30] - 3.25735682321217e-6*U[29]**3*U[31] - 0.000260589700996678*U[31] + 4.36192251724445e-5
    U_out[32] = -8.470329472543e-22*U[10]**4 - 1.25605469675209e-5*U[10]**3*U[12] + 2.05688778355549e-5*U[10]**3*U[14] - 2.51210939350418e-6*U[10]**3*U[17] - 5.42101086242752e-20*U[10] - 0.000179437180657833*U[12] + 0.000293842414478535*U[14] - 3.58874361315667e-5*U[17] - 1.90560783367934*U[21]*U[35] + 0.952803916839669*U[21]*U[37] - 1.46939216632066*U[22]*U[34] - 0.304205875259503*U[22]*U[36] + 1.21682350103801*U[23]*U[35] - 0.608411750519007*U[23]*U[37] + 3.33481370893884*U[24]*U[34] + 1.52102937629752*U[24]*U[36] + 1.46939216632066*U[25]*U[31] - 3.33481370893884*U[25]*U[33] + 1.90560783367934*U[26]*U[30] - 1.21682350103801*U[26]*U[32] + 0.304205875259503*U[27]*U[31] - 1.52102937629752*U[27]*U[33] - 0.952803916839669*U[28]*U[30] + 0.608411750519007*U[28]*U[32] + 1.05879118406788e-22*U[29]**4 - 4.2351647362715e-22*U[29]**3*U[30] - 3.25735682321217e-6*U[29]**3*U[32] - 0.000260589700996678*U[32]
    U_out[33] = 1.11521173268629e-5*U[10]**3*U[11] + 0.000159316668029834*U[11] - 1.875*U[21]*U[36] - 2.25*U[22]*U[35] - 2.625*U[23]*U[34] + 2.625*U[25]*U[32] + 2.25*U[26]*U[31] + 1.875*U[27]*U[30] - 3.25735682321217e-6*U[29]**3*U[33] - 0.000260589700996678*U[33] + 1.74476900689778e-5
    U_out[34] = 3.64823964199763e-5*U[10]**3*U[13] + 0.000521179401993355*U[13] + 1.125*U[21]*U[31] - 1.125*U[22]*U[30] + 1.875*U[22]*U[32] - 1.875*U[23]*U[31] + 2.625*U[23]*U[33] - 2.625*U[24]*U[32] - 3.25735682321217e-6*U[29]**3*U[34] - 0.000260589700996678*U[34]
    U_out[35] = 3.64823964199763e-5*U[10]**3*U[16] + 0.000521179401993355*U[16] + 1.5*U[21]*U[32] + 2.25*U[22]*U[33] - 1.5*U[23]*U[30] - 2.25*U[24]*U[31] - 3.25735682321217e-6*U[29]**3*U[35] - 0.000260589700996678*U[35]
    U_out[36] = -0.375*U[21]*U[31] + 1.875*U[21]*U[33] + 0.375*U[22]*U[30] - 1.875*U[24]*U[30] - 3.25735682321217e-6*U[29]**3*U[36] - 0.000260589700996678*U[36]
    U_out[37] = -0.75*U[21]*U[32] + 0.75*U[23]*U[30] - 3.25735682321217e-6*U[29]**3*U[37] - 0.000260589700996678*U[37]
    return U_out

In [ ]:
from qgs.integrators.integrator import RungeKuttaIntegrator, RungeKuttaTglsIntegrator
import matplotlib.pyplot as plt

In [ ]:
integrator = RungeKuttaIntegrator()
integrator.set_func(f)

In [ ]:
integrator_num = RungeKuttaIntegrator()
integrator_num.set_func(f_num)

In [ ]:
ic = np.array([
    4.64467907e-02, -2.25054437e-02, -3.51762104e-02,  4.65057475e-03,
   -5.08842464e-03,  1.92789965e-02,  3.47849539e-03,  1.86038254e-02,
   -2.99742925e-03,  8.85862363e-03,  1.53577181e+00,  4.29584574e-02,
   -1.88143295e-02, -1.15114614e-02,  1.24843108e-03, -3.61467108e-03,
    2.81459020e-03,  5.59630451e-03,  3.56804517e-03, -1.52654934e-03,
    3.09046486e-03,  1.18185262e-06,  6.76756261e-04,  4.21981618e-06,
    1.06290914e-05, -1.64056994e-05,  3.40932989e-05,  1.90943692e-05,
    1.08189600e-05,  3.16661484e+00, -3.06749318e-04,  1.82036792e-01,
   -1.33924039e-04,  9.77967977e-03, -5.58853689e-03,  2.33218135e-02,
    3.45971396e-05, -5.88894363e-05
])

In [ ]:
%%time
integrator.integrate(0., 100000., 0.1, ic=ic, write_steps=5)
reference_time, reference_traj = integrator.get_trajectories()

In [ ]:
%%time
integrator_num.integrate(0., 100000., 0.1, ic=ic, write_steps=5)
reference_time_num, reference_traj_num = integrator_num.get_trajectories()

In [ ]:
varx = 22
vary = 31
plt.figure(figsize=(10, 8))

plt.plot(reference_traj[varx], reference_traj[vary], marker='o', ms=0.07, ls='')
plt.plot(reference_traj_num[varx], reference_traj_num[vary], marker='o', ms=0.07, ls='')

plt.xlabel('$'+model_parameters.latex_var_string[varx]+'$')
plt.ylabel('$'+model_parameters.latex_var_string[vary]+'$');

In [ ]:
reference_traj.shape

In [ ]:
error = reference_traj - reference_traj_num

In [ ]:
error[:, 1]